Intiate haystack database

In [6]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

In [7]:
document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

In [8]:
import requests
res = requests.get('http://localhost:9200/_cluster/health')
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 4,
 'active_shards': 4,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 66.66666666666666}

In [9]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

2022-07-13 17:56:48.190524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-13 17:56:48.190728: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-13 17:56:48.190754: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (serapf-B450-AORUS-M): /proc/driver/nvidia/version does not exist
2022-07-13 17:56:48.192630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [10]:
from haystack.retriever.sparse import BM25Retriever

In [11]:
retriever = BM25Retriever(document_store)

In [335]:
query = "When can i use trigonometric?"

In [336]:
responses = retriever.retrieve(query)

In [337]:
responses

[<Document: {'content': 'Trigonometry can be hard at first, but after some practise you will master it! Here are some trigonometry tips: label the hypotenuse, adjacent and opposite on your triangle to help you figure out what identity to use, and remember the mnemonic SOHCAHTOA for the trigonometric relationships!', 'content_type': 'text', 'score': 0.7048246969267418, 'meta': {}, 'embedding': None, 'id': '2d06271e0c86e212586b7f081dd5963'}>,
 <Document: {'content': "In the 3rd century BC, Hellenistic mathematicians such as Euclid and Archimedes studied the properties of chords and inscribed angles in circles, and they proved theorems that are equivalent to modern trigonometric formulae, although they presented them geometrically rather than algebraically. In 140 BC, Hipparchus (from Nicaea, Asia Minor) gave the first tables of chords, analogous to modern tables of sine values, and used them to solve problems in trigonometry and spherical trigonometry.[11] In the 2nd century AD, the Grec

In [338]:
answers = []

for response in responses:
    QA_input = {
        'question': query,
        'context': response.to_dict()['content']
    }
    
    answers.append(nlp(QA_input))
    

In [339]:
answers

[{'score': 0.4440689980983734,
  'start': 39,
  'end': 58,
  'answer': 'after some practise'},
 {'score': 0.000796776614151895,
  'start': 7,
  'end': 21,
  'answer': '3rd century BC'},
 {'score': 0.045763708651065826,
  'start': 257,
  'end': 290,
  'answer': 'directly in a trigonometric table'},
 {'score': 0.05834931135177612,
  'start': 452,
  'end': 492,
  'answer': 'if the distance between the tree and boy'},
 {'score': 0.497750461101532,
  'start': 256,
  'end': 319,
  'answer': 'when the variable inside the trigonometric function is negative'},
 {'score': 0.0012609099503606558,
  'start': 550,
  'end': 597,
  'answer': 'to determine the sign, we can use the CAST Rule'},
 {'score': 0.0593794509768486,
  'start': 323,
  'end': 336,
  'answer': 'all around us'},
 {'score': 0.0033235102891921997,
  'start': 213,
  'end': 244,
  'answer': 'special triangles and CAST rule'},
 {'score': 0.009226152673363686,
  'start': 17,
  'end': 28,
  'answer': 'when θ is 0'},
 {'score': 0.015944087

In [340]:
finalAnswer = max(answers, key=lambda x:x['score'])['answer']

In [341]:
finalAnswer

'when the variable inside the trigonometric function is negative'

In [344]:
while True:
    query = input()
    responses = retriever.retrieve(query)
    
    answers = []

    for response in responses:
        QA_input = {
            'question': query,
            'context': response.to_dict()['content']
        }

        answers.append(nlp(QA_input))
    
    finalAnswer = max(answers, key=lambda x:x['score'])['answer']
    
    print(finalAnswer)
    

what is hatsune miku?
a Vocaloid software voicebank
when was hatsune miku created?
2007


KeyboardInterrupt: Interrupted by user